In [73]:
# -*- coding: utf-8 -*-
import warnings
import os.path
import pickle
import datetime
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso,Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_pipeline

import ow_f01, cxmn_train, cxmn_predict, predict_mean
from code.refactor.common import loadSettingsFromYamlFile,save_object,object2Float,get_column_by_type

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [74]:
p1_used_header = ['productkey', 'promotionkey', 'startdatetime', 'enddatetime', 'jdprice', 'syntheticgrossprice', 'promotiondesc', 'promotiondesc_flag', 'promotiontype', 'promotionsubtype',
                'areatypearray', 'tokenflag', 'directdiscount_discount', 'directdiscount_availabilitynumber', 'bundle_subtype1_threshold', 'bundle_subtype1_giveaway',
                'bundle_subtype4_threshold1', 'bundle_subtype4_giveaway1', 'bundle_subtype4_threshold2', 'bundle_subtype4_giveaway2', 'bundle_subtype4_threshold3',
                'bundle_subtype4_giveaway3', 'bundle_subtype2_threshold', 'bundle_subtype2_giveaway', 'bundle_subtype2_maximumgiveaway', 'bundle_subtype15_thresholdnumber1',
                'bundle_subtype15_giveawayrate1', 'bundle_subtype15_thresholdnumber2', 'bundle_subtype15_giveawayrate2', 'bundle_subtype15_thresholdnumber3',
                'bundle_subtype15_giveawayrate3', 'bundle_subtype6_thresholdnumber', 'bundle_subtype6_freenumber', 'suit_maxvaluepool', 'suit_minvaluepool', 'suit_avgvaluepool',
                'suit_discount', 'directdiscount_saleprice', 'bundle_subtype1_percent', 'bundle_subtype4_percent', 'bundle_subtype2_percent', 'bundle_subtype15_percent',
                'bundle_subtype6_percent', 'suit_percent', 'allpercentdiscount', 'mainproductkey', 'hierarchylevel3key', 'createdate', 'statuscode', 'dt']
p2_used_header = ['ProductKey', 'Date', 'HierarchyLevel3Key', 'PromotionCount', 'bundlecount', 'MaxDiscount', 'MinDiscount', 'AvgDiscount', 'MaxSyntheticDiscountA',
				         'MinSyntheticDiscountA', 'AvgSyntheticDiscountA', 'MaxBundleDiscount', 'MinBundleDiscount', 'AvgBundleDiscount', 'MaxDirectDiscount', 'MinDirectDiscount',
				         'AvgDirectDiscount', 'MaxFreegiftDiscount', 'MinFreegiftDiscount', 'AvgFreegiftDiscount', 'SyntheticGrossPrice', 'promotionkey', 'promotiontype',
				         'promotionsubtype', 'syntheticgrossprice_vb', 'jdprice', 'syntheticdiscounta_vb', 'durationinhours', 'daynumberinpromotion', 'bundleflag', 'directdiscountflag',
				         'freegiftflag', 'suitflag', 'numberproducts', 'numberhierarchylevel1', 'numberhierarchylevel2', 'numberhierarchylevel3', 'strongmark', 'stockprice', 'dt']
suffix = '.da'
item = 'p1'
for_what = ['train', 'predict']

In [75]:
class GaussianFeatures(BaseEstimator, TransformerMixin):
    """Uniformly spaced Gaussian features for one-dimensional input"""

    def __init__(self, N, width_factor=2.0):
        self.N = N
        self.width_factor = width_factor

    @staticmethod
    def _gauss_basis(x, y, width, axis=None):
        arg = (x - y) / width
        return np.exp(-0.5 * np.sum(arg ** 2, axis))

    def fit(self, X, y=None):
        # create N centers spread along the data range
        self.centers_ = np.linspace(X.min(), X.max(), self.N)
        self.width_ = self.width_factor * (self.centers_[1] - self.centers_[0])
        return self

    def transform(self, X):
        return self._gauss_basis(X[:, :, np.newaxis], self.centers_,
                                 self.width_, axis=1)

In [76]:
def reduce_df_mem_usage(df):
    # memery now
    start_mem_usg = df.memory_usage().sum() / 1024 ** 2
    print("Memory usage of the dataframe is :", start_mem_usg, "MB")
    
    #np.nan will be handled as float
    NAlist = []
    for col in df.columns:
        # filter object type
        if (df[col].dtypes == np.float64):
            df[col] = df[col].astype(np.float32)
            continue
        if (df[col].dtypes != object)&(df[col].dtypes != 'datetime64[ns]'):
            
            print("**************************")
            print("columns: %s"%col)
            print("dtype before: %s"%df[col].dtype)
            
            # if int or not
            isInt = False
            mmax = df[col].max()
            mmin = df[col].min()
            
            # Integer does not support NA, therefore Na needs to be filled
            if not np.isfinite(df[col]).all():
                NAlist.append(col)
                #continue
                df[col].fillna(-999, inplace=True) # fill -999
                
            # test if column can be converted to an integer
            asint = df[col].fillna(0).astype(np.int64)
            result = np.fabs(df[col] - asint)
            result = result.sum()
            if result < 0.01: # absolute error < 0.01,then could be saw as integer
                isInt = True
            
            # make interger / unsigned Integer datatypes
            if isInt:
                if mmin >= 0: # min>=0, then unsigned integer
                    if mmax <= np.iinfo(np.uint8).max:
                        df[col] = df[col].astype(np.uint8)
                    elif mmax <= np.iinfo(np.uint16).max:
                        df[col] = df[col].astype(np.uint16)
                    elif mmax <= np.iinfo(np.uint32).max:
                        df[col] = df[col].astype(np.uint32)
                    else:
                        df[col] = df[col].astype(np.uint64)
                else:
                    if mmin > np.iinfo(np.int8).min and mmax < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif mmin > np.iinfo(np.int16).min and mmax < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif mmin > np.iinfo(np.int32).min and mmax < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif mmin > np.iinfo(np.int64).min and mmax < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)  
            df.replace(-999, np.nan, inplace=True)
            print("dtype after: %s"%df[col].dtype)
            print("********************************")
    print("___MEMORY USAGE AFTER CONVERSION:___")
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return df

In [77]:
def statitics_mape(new_df_final):
		new_df_sku = new_df_final.groupby('ProductKey').sum().reset_index()
		print "ensemble pred sum : %f"%(new_df_sku.ypred_mean_promo_new.sum())
		print "raw pred sum :      %f"%(new_df_sku.ypred_raw.sum())
		print "actual sum:         %f"%(new_df_sku.salesForecast.sum())

		print "raw pred residual:      %f"%(np.sum(np.abs(new_df_sku.ypred_raw - new_df_sku.salesForecast)))
		print "ensemble pred residual: %f"%(np.sum(np.abs(new_df_sku.ypred_mean_promo_new - new_df_sku.salesForecast)))

		print "raw pred mape: %f"%(np.sum(np.abs(new_df_sku.ypred_raw - new_df_sku.salesForecast)) / new_df_sku.salesForecast.sum())
		print "ensemble mape: %f"%(np.sum(np.abs(new_df_sku.ypred_mean_promo_new - new_df_sku.salesForecast)) / new_df_sku.salesForecast.sum())

In [78]:
def get_online_history_data():
    p1 = pd.read_csv('/home/ubuntu/sunjiadong/promotion_offline/tmp/data/7054_p1.da', sep='\t', header=None)
    p2 = pd.read_csv('/home/ubuntu/sunjiadong/promotion_offline/tmp/data/7054_p2.da', sep='\t', header=None)
    ts = pd.read_csv('/home/ubuntu/sunjiadong/promotion_offline/tmp/data/7054_ts.da', sep='\t', header=None)
    
    # p1 = pd.read_csv('/home/ubuntu/sunjiadong/promotion_offline/tmp/data/shishang/12029/12029_p1.da',sep='\t',header=None)
    # p2 = pd.read_csv('/home/ubuntu/sunjiadong/promotion_offline/tmp/data/shishang/12029/12029_p2.da',sep='\t',header=None)
    # ts = pd.read_csv('/home/ubuntu/sunjiadong/promotion_offline/tmp/data/shishang/12029/12029_ts.da',sep='\t',header=None)
    return p1,p2,ts


In [79]:
scenarioSettingsPath = 'code/refactor/ow_scenario.yaml'
scenario = loadSettingsFromYamlFile(scenarioSettingsPath)
area_rdc_map = pd.read_csv('/home/ubuntu/yulong/promotion_offline/tmp/ow_deploy_single/area_rdc_mapping.csv')
holidays_df=pd.read_csv('/home/ubuntu/yulong/promotion_offline/tmp/ow_deploy_single/holidays.csv')
seasonality_df = pd.read_csv('tmp/data/870_season.csv', parse_dates=['Date'])

In [80]:
cate = 7054
pred_date = pd.to_datetime('2018-02-01')
scenario['lookforwardPeriodDays'] = 10

#cate = 12029
#pred_date = pd.to_datetime('2018-05-07')
#scenario['lookforwardPeriodDays'] = 7


In [81]:
p1_df,p2_df,ts_df = get_online_history_data()

In [82]:
p2_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42150 entries, 0 to 42149
Data columns (total 40 columns):
0     42150 non-null int64
1     42150 non-null object
2     42150 non-null int64
3     42150 non-null int64
4     42150 non-null int64
5     42150 non-null float64
6     42150 non-null float64
7     42150 non-null float64
8     42150 non-null object
9     42150 non-null object
10    42150 non-null object
11    42150 non-null float64
12    42150 non-null float64
13    42150 non-null float64
14    42150 non-null float64
15    42150 non-null float64
16    42150 non-null float64
17    42150 non-null float64
18    42150 non-null float64
19    42150 non-null float64
20    42150 non-null object
21    42150 non-null object
22    42150 non-null object
23    42150 non-null object
24    42150 non-null object
25    42150 non-null object
26    42150 non-null object
27    42150 non-null object
28    42150 non-null object
29    42150 non-null object
30    42150 non-null object
31    42150 non

In [83]:
p1_df.columns = p1_used_header
p2_df.columns = p2_used_header


In [84]:
###handle p2
p2_df['Date'] = pd.to_datetime(p2_df['Date'])
p2_df['dt'] = pd.to_datetime(p2_df['dt'])
p2_df.replace('null', np.nan, inplace=True)
p2_df.replace('None', np.nan, inplace=True)
p2_df.replace(-999, np.nan, inplace=True)
p2_df.drop_duplicates(inplace=True)

# Convert Object -> float
if 'object' in p2_df.dtypes.values:
    obj_cols = get_column_by_type(p2_df,'object')
    object2Float(p2_df,obj_cols)


convert to Float columns:
MaxSyntheticDiscountA
convert to Float columns:
MinSyntheticDiscountA
convert to Float columns:
AvgSyntheticDiscountA
convert to Float columns:
SyntheticGrossPrice
convert to Float columns:
promotionkey
convert to Float columns:
promotiontype
convert to Float columns:
promotionsubtype
convert to Float columns:
syntheticgrossprice_vb
convert to Float columns:
jdprice
convert to Float columns:
syntheticdiscounta_vb
convert to Float columns:
durationinhours
convert to Float columns:
daynumberinpromotion
convert to Float columns:
bundleflag
convert to Float columns:
directdiscountflag
convert to Float columns:
freegiftflag
convert to Float columns:
suitflag


In [85]:
p2_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42150 entries, 0 to 42149
Data columns (total 40 columns):
ProductKey                42150 non-null int64
Date                      42150 non-null datetime64[ns]
HierarchyLevel3Key        42150 non-null int64
PromotionCount            42150 non-null int64
bundlecount               42150 non-null int64
MaxDiscount               42150 non-null float64
MinDiscount               42150 non-null float64
AvgDiscount               42150 non-null float64
MaxSyntheticDiscountA     34264 non-null float64
MinSyntheticDiscountA     34264 non-null float64
AvgSyntheticDiscountA     34264 non-null float64
MaxBundleDiscount         42150 non-null float64
MinBundleDiscount         42150 non-null float64
AvgBundleDiscount         42150 non-null float64
MaxDirectDiscount         42150 non-null float64
MinDirectDiscount         42150 non-null float64
AvgDirectDiscount         42150 non-null float64
MaxFreegiftDiscount       42150 non-null float64
MinFreegif

In [86]:
def handle_f01(p1_df, for_what, area_rdc_map, pred_date, scenario):
    for fw in for_what:
    	print "output and save: %s_p1_%s"%(str(cate),fw)
    	train_pred_gate = fw   # 'train'
    	f01_output = ow_f01.generate_f01_promo(area_rdc_map, p1_df, scenario, train_pred_gate, ForecastStartDate=pred_date)
    	if train_pred_gate == 'train':
    		train_p1_df = f01_output
    	else:
    		predict_p1_df = f01_output
    return train_p1_df, predict_p1_df
train_p1_df,predict_p1_df = handle_f01(p1_df, for_what, area_rdc_map, pred_date, scenario)


output and save: 7054_p1_train
convert to Float columns:
allpercentdiscount
convert to Float columns:
bundle_subtype15_giveawayrate1
convert to Float columns:
bundle_subtype15_giveawayrate2
convert to Float columns:
bundle_subtype15_giveawayrate3
convert to Float columns:
bundle_subtype15_percent
convert to Float columns:
bundle_subtype15_thresholdnumber1
convert to Float columns:
bundle_subtype15_thresholdnumber2
convert to Float columns:
bundle_subtype15_thresholdnumber3
convert to Float columns:
bundle_subtype1_giveaway
convert to Float columns:
bundle_subtype1_percent
convert to Float columns:
bundle_subtype1_threshold
convert to Float columns:
bundle_subtype2_giveaway
convert to Float columns:
bundle_subtype2_maximumgiveaway
convert to Float columns:
bundle_subtype2_percent
convert to Float columns:
bundle_subtype2_threshold
convert to Float columns:
bundle_subtype4_giveaway1
convert to Float columns:
bundle_subtype4_giveaway2
convert to Float columns:
bundle_subtype4_giveaway3
co

In [87]:
print train_p1_df.shape
print predict_p1_df.shape

(76155, 23)
(21902, 23)


In [88]:
ts_df.columns = ['Date', 'ind', 'RDCKey', 'ProductKey', 'HierarchyLevel1Key', 'HierarchyLevel2Key', 'HierarchyLevel3Key', 'brand_code', 'sales', 'priceAfterDiscount', 'jd_prc', 'vendibility', 'counterState', 'salesForecast', 'reserveState', 'stockQuantity', 'utc_flag']
ts_df['Date'] = pd.to_datetime(ts_df['Date'])

ts_df.replace('null', np.nan, inplace=True)
ts_df.replace(-999, np.nan, inplace=True)
ts_df.replace('None', np.nan, inplace=True)
ts_to_float_col = ['RDCKey', 'ProductKey', 'HierarchyLevel1Key', 'HierarchyLevel2Key', 'HierarchyLevel3Key', 'brand_code', 'sales', 'priceAfterDiscount', 'jd_prc', 'vendibility', 'counterState', 'salesForecast', 'reserveState', 'stockQuantity', 'utc_flag']
if 'object' in ts_df[ts_to_float_col].dtypes.values:
    object2Float(ts_df,ts_to_float_col)
levels = ['HierarchyLevel3Key','ProductKey','RDCKey','Date']
ts_df.sort_values(levels, ascending=[True]*len(levels), inplace=True)
ts_df= ts_df.reset_index(drop=True)



convert to Float columns:
RDCKey
convert to Float columns:
ProductKey
convert to Float columns:
HierarchyLevel1Key
convert to Float columns:
HierarchyLevel2Key
convert to Float columns:
HierarchyLevel3Key
convert to Float columns:
brand_code
convert to Float columns:
sales
convert to Float columns:
priceAfterDiscount
convert to Float columns:
jd_prc
convert to Float columns:
vendibility
convert to Float columns:
counterState
convert to Float columns:
salesForecast
convert to Float columns:
reserveState
convert to Float columns:
stockQuantity
convert to Float columns:
utc_flag


In [89]:
ts_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 972720 entries, 0 to 972719
Data columns (total 17 columns):
Date                  972720 non-null datetime64[ns]
ind                   972720 non-null int64
RDCKey                972720 non-null float64
ProductKey            972720 non-null float64
HierarchyLevel1Key    972720 non-null float64
HierarchyLevel2Key    972720 non-null float64
HierarchyLevel3Key    972720 non-null float64
brand_code            972720 non-null float64
sales                 972720 non-null float64
priceAfterDiscount    228354 non-null float64
jd_prc                228354 non-null float64
vendibility           972720 non-null float64
counterState          972720 non-null float64
salesForecast         972720 non-null float64
reserveState          972720 non-null float64
stockQuantity         972720 non-null float64
utc_flag              972720 non-null float64
dtypes: datetime64[ns](1), float64(15), int64(1)
memory usage: 126.2 MB


In [90]:
def train_model(area_rdc_map, train_p1_df, p2_df, ts_df, scenario, holidays_df, seasonality_df, pred_date):
	seasonality_df_train = seasonality_df.copy()
	model,feature=cxmn_train.train(area_rdc_map,train_p1_df,p2_df,ts_df,scenario,holidays_df,seasonality_df_train,process_f01_flag=False,mode='dev',ForecastStartDate=pred_date)
	#feature.to_csv(os.path.join(result_path, str(cate)+'/'+str(cate)+'_train_feature.csv'),index=False)
	#save_object(model, os.path.join(result_path, str(cate)+'/'+str(cate)+'_train_model.pkl'))
	return model, feature
model, feature = train_model(area_rdc_map, train_p1_df, p2_df, ts_df, scenario, holidays_df, seasonality_df, pred_date)


convert to Int columns:
Holiday
convert to Int columns:
Ind_1111_pre
convert to Int columns:
Ind_1111
convert to Int columns:
Ind_1111_post
convert to Int columns:
Ind_618_pre
convert to Int columns:
Ind_618
convert to Int columns:
Ind_618_post
convert to Int columns:
Ind_1212
use cart feature?
False
| Add date features...
| - Add month of the year...
| - Add day of the week...
| Calculating national rolling features...
('| - Rolling value:', 'xxxHashColumn')
| - Rolling 14 mean...
| - Rolling 7 mean...
| - Rolling 5 mean...
| - Rolling 3 mean...
| - Rolling 2 mean...
| - Rolling 1 mean...
| - Rolling 14 median...
| - Rolling 7 median...
| - Rolling decay 28 mean...
| - Rolling decay 14 mean...
| - Rolling decay 7 mean...
| - Rolling decay 3 mean...
| - Added rolling features in 10.030496 seconds 

| Calculating  rolling features...
('| - Rolling value:', 'OrderNonOutlierVolume')
| - Rolling 360 mean...
| - Rolling 180 mean...
| - Rolling 90 mean...
| - Rolling 28 mean...
| - Rolling 1

| - Shift 90...
| - Shift 30...
| - Shift 7...
| - Added rolling features in 2.164192 seconds 

featuresDf shape
(213245, 141)
rdc
3.0
rdc
4.0
rdc
5.0
rdc
6.0
rdc
9.0
rdc
10.0
rdc
316.0
rdc
772.0


In [91]:
def predict_q_pred(area_rdc_map, predict_p1_df, p2_df, ts_df, scenario, holidays_df, seasonality_df, pred_date, model):
	seasonality_df_test = seasonality_df.copy()
	q_pred_result,df_fut=cxmn_predict.predict(area_rdc_map,predict_p1_df,p2_df,ts_df,scenario,holidays_df,model,seasonality_df_test,process_f01_flag=False,mode='dev',ForecastStartDate=pred_date)
	#q_pred_result.to_csv('tmp/data/test_0705/'+'result_'+str(cate)+'.csv',index=False)
	return q_pred_result, df_fut
q_pred_result, df_fut = predict_q_pred(area_rdc_map, predict_p1_df, p2_df, ts_df, scenario, holidays_df, seasonality_df, pred_date, model)


convert to Int columns:
Holiday
convert to Int columns:
Ind_1111_pre
convert to Int columns:
Ind_1111
convert to Int columns:
Ind_1111_post
convert to Int columns:
Ind_618_pre
convert to Int columns:
Ind_618
convert to Int columns:
Ind_618_post
convert to Int columns:
Ind_1212
| Add date features...
| - Add month of the year...
| - Add day of the week...
| - group by SyntheticPromotionSubType...


In [52]:
feature.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 213245 entries, 0 to 213244
Columns: 141 entries, Date to bd_discount_sgp_wgt
dtypes: datetime64[ns](3), float64(127), int64(10), object(1)
memory usage: 235.9 MB


In [92]:
train_feature_df_new = feature

In [93]:
train_feature_df_new = reduce_df_mem_usage(feature)

('Memory usage of the dataframe is :', 231, 'MB')
**************************
columns: Holiday
dtype before: int64
dtype after: uint8
********************************
**************************
columns: Ind_1111_pre
dtype before: int64
dtype after: uint8
********************************
**************************
columns: Ind_1111
dtype before: int64
dtype after: uint8
********************************
**************************
columns: Ind_1111_post
dtype before: int64
dtype after: uint8
********************************
**************************
columns: Ind_618_pre
dtype before: int64
dtype after: uint8
********************************
**************************
columns: Ind_618
dtype before: int64
dtype after: uint8
********************************
**************************
columns: Ind_618_post
dtype before: int64
dtype after: uint8
********************************
**************************
columns: Ind_1212
dtype before: int64
dtype after: uint8
********************************


In [94]:
def predict_q_mean(area_rdc_map, predict_p1_df, p2_df, ts_df, scenario, holidays_df, seasonality_df, pred_date, model, train_feature_df_new):
	seasonality_df_mean = seasonality_df.copy()
	q_mean_result,df_fut_mean = predict_mean.predict(area_rdc_map,predict_p1_df,p2_df,ts_df,scenario,holidays_df,model,seasonality_df_mean,process_f01_flag=False,mode='dev',ForecastStartDate=pred_date,train_feature=train_feature_df_new)
	#q_mean_result.to_csv('tmp/data/shishang/result/'+'result_'+str(cate)+'_mean.csv',index=False)
	return q_mean_result, df_fut_mean
q_mean_result, df_fut_mean = predict_q_mean(area_rdc_map, predict_p1_df, p2_df, ts_df, scenario, holidays_df, seasonality_df, pred_date, model, train_feature_df_new)


convert to Int columns:
Holiday
convert to Int columns:
Ind_1111_pre
convert to Int columns:
Ind_1111
convert to Int columns:
Ind_1111_post
convert to Int columns:
Ind_618_pre
convert to Int columns:
Ind_618
convert to Int columns:
Ind_618_post
convert to Int columns:
Ind_1212
| Add date features...
| - Add month of the year...
| - Add day of the week...
| - group by SyntheticPromotionSubType...


In [95]:
def get_actual_sales(ts_df, pred_date, scenario):
	simplified_ts_df = ts_df[ts_df.Date.between(pred_date, pd.to_datetime(pred_date)+\
                       pd.DateOffset(days=scenario['lookforwardPeriodDays']-1))]
	return simplified_ts_df
simplified_ts_df = get_actual_sales(ts_df, pred_date, scenario)


In [96]:
simplified_ts_df.ProductKey	.drop_duplicates().shape

(155,)

In [97]:
ForecastStartDate = pd.to_datetime(pred_date)
DataStartDate = ForecastStartDate - datetime.timedelta(days=scenario['lookbackPeriodDays'])
PredictEndDate = ForecastStartDate + datetime.timedelta(days=(scenario['lookforwardPeriodDays']-1))

actual = simplified_ts_df
actual.Date = pd.to_datetime(actual.Date)
actual.RDCKey = actual.RDCKey.astype(float)

list_keys = ['Date','RDCKey','ProductKey']
feat_cols = ['dd_price_weighted','bd_price_weighted','dd_price_weighted_x','bd_price_weighted_x','SyntheticGrossPrice']
exclu_promo_features = ['strongmark','flashsale_ind','dd_ind','bundle_ind','bundle_buy199get100_ind','suit_ind','freegift_ind']


In [98]:
def get_raw_test_df(raw_pred, mean_pred, keys, feat_cols):

	raw = raw_pred  ###bottomup forecast
	raw = raw[keys + ['salesForecast','ypred']]
	raw.rename(columns={'ypred':'ypred_raw'},inplace=True)
	raw.drop('salesForecast',axis=1,inplace=True)

	mean_df = mean_pred
	mean_df = mean_df[keys + feat_cols + ['salesForecast','ypred']]
	mean_df.rename(columns={'ypred':'ypred_mean_promo'},inplace=True)
	mean_df.drop('salesForecast',axis=1,inplace=True)

	new_df = raw.merge(mean_df,on=list_keys)
	new_df.Date = pd.to_datetime(new_df.Date)
	new_df = pd.merge(new_df, actual[list_keys+['salesForecast']], how='left',on = list_keys)

	return new_df
new_df = get_raw_test_df(q_pred_result, q_mean_result, list_keys, feat_cols)


/home/ubuntu/lijingjie/ljj/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ubuntu/lijingjie/ljj/lib/python2.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [99]:
new_df.head()

,Date,RDCKey,ProductKey,ypred_raw,dd_price_weighted,bd_price_weighted,dd_price_weighted_x,bd_price_weighted_x,SyntheticGrossPrice,ypred_mean_promo,salesForecast
0,2018-02-01,3.0,255778,20.571106,153.427719,152.8116,149.0,149.0,149.0,10.873569,26.0
1,2018-02-02,3.0,255778,20.921082,153.427719,152.8116,149.0,149.0,149.0,10.873569,29.0
2,2018-02-03,3.0,255778,20.921082,153.427719,152.8116,149.0,149.0,149.0,10.873569,27.0
3,2018-02-04,3.0,255778,20.921082,153.427719,152.8116,149.0,149.0,149.0,10.873569,21.0
4,2018-02-05,3.0,255778,23.651291,153.427719,152.8116,149.0,149.0,149.0,13.065337,21.0


In [100]:
update_cols = list(set(scenario['promo_feature_cols'])- set(exclu_promo_features))
need_cols = ['Date','RDCKey','ProductKey','HierarchyLevel3Key'] + update_cols
groupkeys = ['RDCKey','ProductKey','HierarchyLevel3Key']
reg_cols = []#['Holiday','Ind_1111_pre','Ind_1111','Ind_1111_post','Ind_618_pre','Ind_618','Ind_618_post','Ind_1212','Month','DayOfWeek',]


In [101]:
def predict_history_mean_and_raw(train_feature_df_new, reg_cols,listkeys, keys, model, update_cols, scenario):
	uses_promo = ['mean','no']
	df = train_feature_df_new
	for use_promo in uses_promo:
	    if use_promo == 'mean':
	        df1 = df[need_cols]
	        promo_feature_cols =  scenario['promo_feature_cols']
	        df11 = df1.groupby(keys)[update_cols].mean().reset_index()
	        df2 = pd.merge(df,df11[keys + update_cols], how='left',on=keys)

	        rename_update_cols = [col+'_y' for col in update_cols]
	        for col in update_cols:
	            df2.rename(columns={col+'_y': col},inplace=True)
	            df2.drop(col+'_x',axis=1,inplace=True)
	        grouped = df2.groupby('RDCKey')
	    else:
	        #histoty bottomup forecast
	        grouped = df.groupby('RDCKey')
	    result_list = []
	    for rdc, history_df in grouped:
	        if rdc in model.keys():
	            this_model = model[rdc]
	        else:
	            continue
	        ''' predict model '''
	        xColumns = scenario['selectedColumns']['features']

	        if 'RDCKey' in xColumns:# 删除季节性,RDCKEY
	            xColumns.remove('skuDecomposedTrend')
	            xColumns.remove('skuDecomposedSeasonal')
	            xColumns.remove('level3DecomposedTrend')
	            xColumns.remove('level3DecomposedSeasonal')
	            xColumns.remove('Curve')
	            xColumns.remove('RDCKey')
	        
	        X_history = history_df[xColumns]

	        history_xtest = xgb.DMatrix(X_history.values, missing=np.NaN )
	        ypred = this_model.predict(history_xtest)
	        history_df['ypred'] =ypred
	        history_df['RDCKey'] = rdc

	        ''' Tuning result '''
	        lanjie = history_df[(history_df.ypred<0)]
	        if len(lanjie)>0:
	            history_df.ix[lanjie.index,'ypred'] = 0
	        result_list.append(history_df)
	    final_result = pd.concat(result_list)
	    if use_promo == 'no':
	        raw_train_df = final_result[listkeys + reg_cols + scenario['promo_feature_cols'] + ['salesForecast','ypred']]
	    else:
	        #use_promo == 'mean':
	        train_df_mean = final_result[listkeys + reg_cols + scenario['promo_feature_cols'] + ['salesForecast','ypred']]
	        train_df_mean.rename(columns={'ypred':'ypred_mean_promo'}, inplace=True)
	return raw_train_df, train_df_mean
raw_train_df, train_df_mean = predict_history_mean_and_raw(train_feature_df_new, reg_cols, list_keys, groupkeys, model, update_cols, scenario)


/home/ubuntu/lijingjie/ljj/lib/python2.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ubuntu/lijingjie/ljj/lib/python2.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [102]:
q_pred_result.Date.max()

Timestamp('2018-02-10 00:00:00')

In [103]:
raw_train_df = pd.merge(raw_train_df, train_df_mean[list_keys+['ypred_mean_promo']], how='left', on=list_keys)
raw_test_df = q_pred_result
raw_test_df = raw_test_df[list_keys + reg_cols + scenario['promo_feature_cols'] + ['salesForecast','ypred']]
raw_test_df.Date = pd.to_datetime(raw_test_df.Date)

used_cols = reg_cols + ['MaxSyntheticDiscountA']  #['MaxBundleDiscount','MaxDirectDiscount','MaxDiscount','MaxSyntheticDiscountA','daynumberinpromotion','PromotionCount']
raw_train_df.Date = pd.to_datetime(raw_train_df.Date)
raw_train_df = raw_train_df[raw_train_df.Date < pred_date]
raw_train_df = raw_train_df[list_keys + reg_cols + scenario['promo_feature_cols'] + ['ypred','ypred_mean_promo']]


In [104]:
input_df = pd.concat([raw_train_df, raw_test_df])

In [105]:
def process_lr_cols(input_df, cols):
	for col in cols:
	    #input_df[col] = input_df.groupby(['RDCKey','ProductKey'])[col].transform(lambda x: x.fillna(method='bfill').fillna(0))
	    #input_df[col] = input_df.groupby(['RDCKey','ProductKey'])[col].transform(lambda x: x.fillna(method='ffill').fillna(0))
	    #input_df = input_df[input_df['MaxSyntheticDiscountA'].between(-1,1)]
	    
	    input_df = input_df[~(input_df[col].isnull())]
	    input_df = input_df[input_df[col].between(-1,1)]
        return input_df

input_df = process_lr_cols(input_df, used_cols)


In [106]:
def lr_promo_simulate(input_df, start_dt, used_cols, listkeys):
	value_type = 'ypred_mean_promo'
	final_df = pd.DataFrame()
	a = 1
	grouped = input_df.groupby(['RDCKey','ProductKey'])
	for (rdc, sku), group in grouped:
	    if group.Date.min() < start_dt and group.Date.max() >= start_dt:
	        print a
	        a = a + 1
	        train_df = group[group.Date < start_dt]
	        test_df = group[group.Date >= start_dt]
	        x_train_df = train_df[used_cols]
	        x_test_df = test_df[used_cols]
	        
	        y_train = train_df['ypred'] - train_df[value_type]
	        y_test = test_df['salesForecast']
	        lm = LinearRegression()
	        lm.fit(x_train_df, y_train)
	        Intercept = lm.intercept_
	        RSquare = lm.score(x_train_df, y_train)
	        lm_predict_result = lm.predict(x_test_df)
	        test_result = pd.DataFrame()
	        for col in listkeys+['salesForecast','ypred']:
	            test_result[col] = test_df[col]
	        test_result['reg_result'] = lm_predict_result

	        ###gaussian###
	        '''
	        if len(x_train_df[used_cols].drop_duplicates()) == 1:
	            test_result = pd.DataFrame()
	            for col in list_keys+['salesForecast','ypred']:
	                test_result[col] = test_df[col]
	            test_result['reg_result'] = y_train.tail().mean()
	        else:
	            lm = make_pipeline(GaussianFeatures(5), Lasso(alpha=0.1))
	            lm.fit(np.array(x_train_df), np.array(y_train))
	            Intercept = lm.steps[1][1].intercept_
	            RSquare = lm.score(np.array(x_train_df), np.array(y_train))
	            lm_predict_result = lm.predict(np.array(x_test_df))
	            test_result = pd.DataFrame()
	            for col in list_keys+['salesForecast','ypred']:
	                test_result[col] = test_df[col]
	            test_result['reg_result'] = lm_predict_result
	        '''
	        ###polynomial###
	        '''
	        pf = PolynomialFeatures(degree=2)
	        pModel = LinearRegression()
	        pModel.fit(pf.fit_transform(x_train_df), y_train)
	        pf_predict_result = pModel.predict(pf.fit_transform(x_test_df))
	        test_result = pd.DataFrame()
	        for col in list_keys+['salesForecast','ypred']:
	            test_result[col] = test_df[col]
	        test_result['reg_result'] = pf_predict_result
	        '''
	        final_df = pd.concat([final_df, test_result])
	return final_df

final_df = lr_promo_simulate(input_df, ForecastStartDate, used_cols, list_keys)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [107]:
input_df.Date.max()

Timestamp('2018-02-09 00:00:00')

In [108]:
final_df.Date.min()

Timestamp('2018-02-01 00:00:00')

In [109]:
final_df.ProductKey.drop_duplicates().shape

(46,)

In [110]:
mean_final = final_df
mean_final.rename(columns={'reg_result':'mean_promo_reg_result'},inplace=True)
final_df.drop('salesForecast',axis=1,inplace=True)
final_df.drop('ypred',axis=1,inplace=True)
new_df_final = new_df.merge(final_df,on=list_keys,how='left')
new_df_final.fillna(0,inplace=True)
new_df_final['ypred_mean_promo_new'] = new_df_final['ypred_mean_promo'] + new_df_final['mean_promo_reg_result']


In [111]:
new_df_final.head(30)

,Date,RDCKey,ProductKey,ypred_raw,dd_price_weighted,bd_price_weighted,dd_price_weighted_x,bd_price_weighted_x,SyntheticGrossPrice,ypred_mean_promo,salesForecast,mean_promo_reg_result,ypred_mean_promo_new
0,2018-02-01,3.0,255778.0,20.571106,153.427719,152.811600,149.0000,149.0000,149.0,10.873569,26.0,1.858400,12.731970
1,2018-02-02,3.0,255778.0,20.921082,153.427719,152.811600,149.0000,149.0000,149.0,10.873569,29.0,1.858400,12.731970
2,2018-02-03,3.0,255778.0,20.921082,153.427719,152.811600,149.0000,149.0000,149.0,10.873569,27.0,1.858400,12.731970
3,2018-02-04,3.0,255778.0,20.921082,153.427719,152.811600,149.0000,149.0000,149.0,10.873569,21.0,1.858400,12.731970
4,2018-02-05,3.0,255778.0,23.651291,153.427719,152.811600,149.0000,149.0000,149.0,13.065337,21.0,1.858400,14.923737
5,2018-02-06,3.0,255778.0,24.609684,153.427719,152.811600,149.0000,149.0000,149.0,13.376189,26.0,1.858400,15.234589
6,2018-02-07,3.0,255778.0,20.990210,153.427719,152.811600,149.0000,149.0000,149.0,11.292673,19.0,1.858400,13.151073
7,2018-02-08,3.0,255778.0,23.995256,153.427719,152.811600,149.0000,149.0000,158.0,12.735791,22.0,1.896542,14.632333
8,2018-02-09,3.0,255778.0,25.687412,153.427719,152.811600,149.0000,149.0000,158.0,12.735791,8.0,1.896542,14.632333
9,2018-02-10,3.0,255778.0,31.010353,153.427719,152.811600,49.6667,49.6667,0.0,12.494842,21.0,0.000000,12.494842


In [112]:
statitics_mape(new_df_final)

ensemble pred sum : 17671.093899
raw pred sum :      19411.119141
actual sum:         21034.000000
raw pred residual:      6840.980298
ensemble pred residual: 7643.863663
raw pred mape: 0.325234
ensemble mape: 0.363405


In [39]:
statitics_mape(new_df_final)

ensemble pred sum : 19398.550781
raw pred sum :      20388.822266
actual sum:         21034.000000
raw pred residual:      5530.731600
ensemble pred residual: 5372.254739
raw pred mape: 0.262942
ensemble mape: 0.255408


In [53]:
new_df_final.ProductKey.drop_duplicates().shape

(4779,)